# Transposed buffer generator

In [42]:
import math 
from datetime import datetime 

In [43]:
####################################################
# Inputs definitions
####################################################
INPUT_N = 4                                     # Input size is a INPUT_N x INPUT_N block of 4x4 sub-blocks
INPUT_COORD_WIDTH = 8                           # Number of bits as input for each block in an input line
INPUT_MACROBLOCK_SIZE = 16                      # Number of pixels in one macroblock (16 = 4x4)
PIXEL_WIDTH = 8                                 # Number of bits for each integer pixel from the reference frame


####################################################
# Interpolator definitions
####################################################
INPUT_LINE_SIZE = 9                             # Input size for ONE line loaded to the buffer (9 integer pixels)
TAPS = 6                                        # Number of taps
FILTERS = 15                                    # Number of filters (= number of outputs expected)
OUT_1ST_INTERP_WIDTH = 11                       # Width for every 15 interpolation output for FIRST PASS
OUT_2ND_INTERP_WIDTH = 14                       # Width for every 15 interpolation output for SECOND PASS


####################################################
# Motion Vector Generator definitions
####################################################
INPUT_COORD_WIDTH = 8                           # Input size for the coordinates X and Y
INPUT_MV_WIDTH = 8                              # Input size for the block motion vectors (MV0 and MV1)
OUTPUT_MV_GEN_WIDTH = 19                        # Output size of the Motion Vector module (4 and 6 parameters are equal)


####################################################
# BUFFER DEFINITIONS
####################################################
LINES = 9
COLUMNS = 4
BUFFER_NAME = 'transposed_buffer'#+str(LINES)+'x'+str(COLUMNS)
INT_REG_NAME = 'reg_internal_buffer'

In [44]:
####################################################
# Buffer variables calculations
####################################################
# Modified date
DATE_MODIF = str(datetime.today().strftime('%d/%m/%Y'))

# Loop to create the module declarations list 
MODULE_LIST = []
IO_LIST = []
for i in range(2*COLUMNS):
    if i < COLUMNS:
        MODULE_LIST.append('\tINPUT_'+str(i)+',')
        IO_LIST.append('  input signed ['+str(OUT_1ST_INTERP_WIDTH-1)+':0] INPUT_'+str(i)+';')
    elif i < 2*COLUMNS-1:
        MODULE_LIST.append('\tOUTPUT_'+str(i-COLUMNS)+',')
        IO_LIST.append('  output wire signed ['+str(OUT_1ST_INTERP_WIDTH*LINES-1)+':0] OUTPUT_'+str(i-COLUMNS)+';')
    else:
        MODULE_LIST.append('\tOUTPUT_'+str(i-COLUMNS))
        IO_LIST.append('  output wire signed ['+str(OUT_1ST_INTERP_WIDTH*LINES-1)+':0] OUTPUT_'+str(i-COLUMNS)+';')

MODULE_LIST_CONCAT = '\n'.join(MODULE_LIST)
IO_LIST_CONCAT = '\n'.join(IO_LIST)

# Wires declarations
INPUT_WIRES = []
for b in range(COLUMNS):
    INPUT_WIRES.append('INPUT_'+str(b))
for d in range(LINES*COLUMNS-COLUMNS):
    INPUT_WIRES.append('out_reg_'+str(d))

OUTPUT_WIRES = []
OUTPUT_WIRES_DECLARATION = []
for c in range(LINES*COLUMNS):
    OUTPUT_WIRES.append('out_reg_'+str(c))
    OUTPUT_WIRES_DECLARATION.append('  wire signed ['+str(OUT_1ST_INTERP_WIDTH-1)+':0] out_reg_'+str(c)+';')
OUTPUT_WIRES_DECLARATION_CONCAT = '\n'.join(OUTPUT_WIRES_DECLARATION)
# print(OUTPUT_WIRES)

# Module instantiations
MODULE_INST = []
for n in range(LINES*COLUMNS):
    MODULE_INST.append(INT_REG_NAME+' R'+str(n)+' (')
    MODULE_INST.append('\t.CLK (WIRE_CLK),')
    MODULE_INST.append('\t.RST_ASYNC_N (WIRE_RST),')
    MODULE_INST.append('\t.WRITE_EN (WIRE_EN),')
    MODULE_INST.append('\t.DATA_IN ('+INPUT_WIRES[n]+'),')
    MODULE_INST.append('\t.DATA_OUT ('+OUTPUT_WIRES[n]+'));\n')
MODULE_INST_CONCAT = '\n'.join(MODULE_INST)

# # Columns wires, assigns and outputs assigns
# COLUMNS_WIRE = []
# COLUMNS_ASSIGN = []
# OUT_CONCAT_LIST = []
# OUT_ASSIGN = []
# for i in range(COLUMNS):
#     COLUMNS_WIRE.append('  wire signed ['+str(LINES*OUT_1ST_INTERP_WIDTH-1)+':0] COLUMN_'+str(i)+';')
#     for lin in range(0,LINES*COLUMNS,COLUMNS):
#         OUT_CONCAT_LIST = [OUTPUT_WIRES[i+lin]] + OUT_CONCAT_LIST
#     OUT_CONCAT_LIST_CONCAT = ', '.join(OUT_CONCAT_LIST[0:LINES])
#     COLUMNS_ASSIGN.append('  assign COLUMN_'+str(i) +' = {'+OUT_CONCAT_LIST_CONCAT+'};')
#     OUT_ASSIGN.append('  assign OUTPUT_'+str(i)+' = COLUMN_'+str(i)+';')
# COLUMNS_WIRE_CONCAT = '\n'.join(COLUMNS_WIRE)    
# COLUMNS_ASSIGN_CONCAT = '\n'.join(COLUMNS_ASSIGN)
# OUT_ASSIGN_CONCAT = '\n'.join(OUT_ASSIGN)

# Columns wires, assigns and outputs assigns
COLUMNS_WIRE = []
COLUMNS_ASSIGN = []
OUT_CONCAT_LIST = []
OUT_ASSIGN = []
for i in range(COLUMNS):
    COLUMNS_WIRE.append('  wire signed ['+str(LINES*OUT_1ST_INTERP_WIDTH-1)+':0] COLUMN_'+str(i)+';')
    for lin in range(0,LINES*COLUMNS,COLUMNS):
        OUT_CONCAT_LIST.append(OUTPUT_WIRES[i+lin])
    OUT_CONCAT_LIST_CONCAT = ', '.join(OUT_CONCAT_LIST[i*LINES:])
    COLUMNS_ASSIGN.append('  assign COLUMN_'+str(i) +' = {'+OUT_CONCAT_LIST_CONCAT+'};')
    OUT_ASSIGN.append('  assign OUTPUT_'+str(i)+' = COLUMN_'+str(i)+';')
COLUMNS_WIRE_CONCAT = '\n'.join(COLUMNS_WIRE)    
COLUMNS_ASSIGN_CONCAT = '\n'.join(COLUMNS_ASSIGN)
OUT_ASSIGN_CONCAT = '\n'.join(OUT_ASSIGN)

# Explanation of positions 
EXPLAIN = '''
/* INPUTS = [
IN8:    [1, 2, 4, 10, 12, 120, 125, 1, 100],
IN7:    [114, 2, 93, 0, 1, 7, 100, 4, 2], 
IN6:    [9, 66, 4, 10, 121, 1, 2, 54, 77], 
IN5:    [1, 3, 33, 5, 8, 12, 13, 1, 100],
IN4:    [124, 112, 2, 99, 41, 4, 2, 6, 8],
IN3:    [5, 1, 0, 44, 47, 106, 124, 1, 29],
IN2:    [0, 55, 97, 25, 66, 68, 44, 7, 4],
IN1:    [22, 64, 7, 4, 0, 15, 22, 20, 10],
IN0:    [13, 14, 16, 127, 1, 88, 14, 6, 0]]

COL0:   [[4, 93, 4, 33, 2, 0, 97, 7, 16], 
COL1:   [10, 0, 10, 5, 99, 44, 25, 4, 127], 
COL2:   [12, 1, 121, 8, 41, 47, 66, 0, 1], 
COL3:   [120, 7, 1, 12, 4, 106, 68, 15, 88]] */'''

# print(MODULE_LIST_CONCAT)
# print(IO_LIST_CONCAT)
# print(INPUT_WIRES)
# print(INPUT_WIRES_DECLARATION_CONCAT)
# print(OUTPUT_WIRES)
# print(OUTPUT_WIRES_DECLARATION_CONCAT)
# print(MODULE_INST_CONCAT)
# print(COLUMNS_WIRE_CONCAT)
# print(OUT_CONCAT_LIST)
# print(COLUMNS_ASSIGN_CONCAT)
# print(OUT_ASSIGN_CONCAT)

In [45]:
####################################################
# Buffer Verilog text
####################################################
buffer_verilog = '''/*-----------------------------------------------------------------------------------
* File: '''+BUFFER_NAME+'''.v
* Date generated: 25/03/2023
* Date modified: '''+DATE_MODIF+'''
* Author: Bruna Suemi Nagai
* Description: Transposed buffer that writes lines from top to bottom. And reads columns from left to right. 
*----------------------------------------------------------------------------------- */
'''+EXPLAIN+'''

module '''+BUFFER_NAME+''' (
\tCLK, 
\tRST_ASYNC_N,
\tWRITE_EN,
'''+MODULE_LIST_CONCAT+'''
);


// ------------------------------------------
// Port mode declaration
// ------------------------------------------
  input CLK;\t\t\t\t\t\t\t\t//Clock
  input RST_ASYNC_N;\t\t\t\t\t\t//Asynchronous reset
  input WRITE_EN;\t\t\t\t\t\t\t//Enable writing for all internal registers
'''+IO_LIST_CONCAT+'''


// ------------------------------------------
// Wires declarations
// ------------------------------------------
  wire WIRE_CLK;
  wire WIRE_RST;
  wire WIRE_EN;

// Outputs from individual registers
'''+OUTPUT_WIRES_DECLARATION_CONCAT+'''

// Columns read as the buffer output
'''+COLUMNS_WIRE_CONCAT+'''


// ------------------------------------------
// Modules instantiation
// ------------------------------------------
'''+MODULE_INST_CONCAT+'''


// ------------------------------------------
// Combinational logic
// ------------------------------------------
  assign WIRE_CLK = CLK;
  assign WIRE_RST = RST_ASYNC_N;
  assign WIRE_EN = WRITE_EN;

  // From columns to output
'''+COLUMNS_ASSIGN_CONCAT+'''


// ------------------------------------------
// Outputs
// ------------------------------------------
'''+OUT_ASSIGN_CONCAT+'''


endmodule // '''+BUFFER_NAME+'''
'''

# print(buffer_verilog)

# Write the buffer file
with open('../verilog/registers/'+BUFFER_NAME+'.v', 'w') as buffer_file:
    buffer_file.write(buffer_verilog)